In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender==1.0.2

In [ ]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

In [ ]:
import gym
from colabgymrender.recorder import Recorder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.optimizers import Adam

In [ ]:
env = gym.make("FrozenLake-v0")
q_table = np.zeros((env.observation_space.n,env.action_space.n))
alpha = 0.1
gamma = 0.95
epsilon = 1.0
rewards = []
i=0
frames=0
while True:
  i+=1
  state = env.reset()
  ep_reward = 0
  done = False
  for b in range(200):
    frames+=1
    if np.random.rand(1)[0] < epsilon:
      action = env.action_space.sample()
    else:
      action = np.argmax(q_table[state])

    state_,r,done,_ = env.step(action)
    ep_reward+=r
    epsilon-= 0.99/100000
    epsilon = max(epsilon,0.1)

    q_table[state,action] = q_table[state,action] + alpha*(r + gamma*np.max(q_table[state_])-q_table[state,action])

    state=state_
    if done:
      break
  rewards.append(ep_reward)
  if sum(rewards[-100:])/100>0.78:
    print("environment is solved after {} episode".format(i))
    break
  if i%10000==0:
    print("episode:",i,"frame:",frames,"mean reward:",sum(rewards[-100:])/100)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

q_table = np.load('/content/gdrive/My Drive/Colab Notebooks/frozenlake.npy')

In [ ]:
# directory = './video'
# env = Recorder(env, directory)
obs = env.reset()
done=False
actions = []
solved=0
for k in range(100):
  obs = env.reset()
  done=False
  while not done:
    action = np.argmax(q_table[obs])
    obs,r,done,_ = env.step(action)
    actions.append(action)
    if r==1:
      solved+=1
print(solved)

82
